In [ ]:
pip install opencv-python imutils


In [1]:
# core python modules
import threading
import winsound

import cv2
import imutils

In [ ]:
import cv2
import imutils
import threading

capture = cv2.VideoCapture(0, cv2.CAP_DSHOW)

capture.set(cv2.CAP_PROP_FRAME_WIDTH, 720)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 500)

_, initial_frame = capture.read()
initial_frame = imutils.resize(initial_frame, width=520)
initial_frame = cv2.cvtColor(initial_frame, cv2.COLOR_BGR2GRAY)
initial_frame = cv2.GaussianBlur(initial_frame, (21, 21), 0)

motion_detected = False
alarm_mode = False
alarm_counter = 0

def beep_alarm():
    global motion_detected
    for _ in range(3):  
        if not alarm_mode:
            break
        print('ALARM')
        
        # winsound.Beep(2500, 1000)  # frequency 2500hz, time 1000ms
    motion_detected = False

while True:
    _, frame = capture.read()
    frame = imutils.resize(frame, width=500)

   
    initial_frame = cv2.resize(initial_frame, (frame.shape[1], frame.shape[0]))

    threshold = None

    if alarm_mode:
        frame_bw = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_bw = cv2.GaussianBlur(frame_bw, (5, 5), 0)

        print("initial_frame dimensions:", initial_frame.shape)
        print("frame_bw dimensions:", frame_bw.shape)

        if initial_frame.shape == frame_bw.shape:
            difference = cv2.absdiff(frame_bw, initial_frame)
            threshold = cv2.threshold(difference, 25, 255, cv2.THRESH_BINARY)[1]
            initial_frame = frame_bw
            if threshold.sum() > 300:
                alarm_counter += 1
            else:
                if alarm_counter > 0:
                    alarm_counter -= 1
        else:
            print("Error: Frame dimensions do not match.")

    cv2.imshow("Cam", threshold if alarm_mode and threshold is not None else frame)

    if alarm_counter > 20 and not motion_detected:
        motion_detected = True
        threading.Thread(target=beep_alarm).start()

    key_pressed = cv2.waitKey(30)
    if key_pressed == ord("s"):
        alarm_mode = not alarm_mode
    elif key_pressed == ord("q"):
        break

capture.release()
cv2.destroyAllWindows()
